In [13]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split

from PIL import Image
import numpy as np
import pandas as pd
import os
import warnings

In [14]:
path = '/Users/utkarsh/Desktop/study/iitj/sem2/ml2/assignment2/101_ObjectCategories'
valid_exts = [".jpg", ".gif", ".png", ".jpeg"]
print ("[%d] CATEGORIES ARE IN \n %s" % (len(os.listdir(path)), path))

[10] CATEGORIES ARE IN 
 /Users/utkarsh/Desktop/study/iitj/sem2/ml2/assignment2/101_ObjectCategories


In [15]:
categories = sorted(os.listdir(path))
ncategories = len(categories)
imgs = []
labels = []
cat_dict = {}
# LOAD ALL IMAGES 
for i, category in enumerate(categories):
    iter = 0
    for f in os.listdir(path + "/" + category):
        if iter == 0:
            ext = os.path.splitext(f)[1]
            if ext.lower() not in valid_exts:
                continue
            fullpath = os.path.join(path + "/" + category, f)
            img = load_img(fullpath, target_size=(224, 224))
            img = img_to_array(img)
            img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
            imgs.append(img) # NORMALIZE IMAGE 
            label_curr = i
            labels.append(label_curr)
            cat_dict[category] = i
        #iter = (iter+1)%10;
print ("Num imgs: %d" % (len(imgs)))
print ("Num labels: %d" % (len(labels)) )
print (ncategories)
print(cat_dict)

Num imgs: 60
Num labels: 60
10
{'accordion': 0, 'airplanes': 1, 'anchor': 2, 'ant': 3, 'bass': 4, 'beaver': 5, 'binocular': 6, 'bonsai': 7, 'buddha': 8, 'butterfly': 9}


In [16]:
seed = 7
np.random.seed(seed)
X_train, X_test, y_train, y_test = train_test_split(imgs, labels, test_size = 0.2) # Change the proportion of training and test
#X_train = np.stack(X_train, axis=0)
#y_train = np.stack(y_train, axis=0)
#X_test = np.stack(X_test, axis=0)
#y_test = np.stack(y_test, axis=0)

#data = {}
#data['categories'] = categories
#data['X_train'] = X_train
#data['y_train'] = y_train
#data['X_test'] = X_test
#data['y_test'] = y_test


In [44]:
warnings.filterwarnings('ignore')

# define model
vgg16 = VGG16(weights='imagenet')
vgg16.summary()

# don't train existing weights
#for layer in vgg16.layers:
#    layer.trainable = False

# Extracrting last leayer from the teh CNN model before the rediction layer
#model = Model(inputs=vgg16.input, outputs=vgg16.get_layer('fc2').output)
temp_labels = []
temp_features = []

for index in range(0, len(X_train)):
    img = X_train[index]
    img = preprocess_input(img)
    # Extracting features
    yhat = vgg16.predict(img)
    # convert the probabilities to class labels
    label = decode_predictions(yhat)
    # retrieve the most likely result, e.g. highest probability
    label = label[0][0]
    # print the classification
    print('%s (%.2f%%)' % (label[1], label[2]*100))



Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_28 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

KeyboardInterrupt: 

In [42]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.layers import Input, Flatten, Dense

model_vgg16_conv = VGG16(weights='imagenet', include_top=False)
#model_vgg16_conv.summary()

#Create your own input format (here 3x200x200)
ip = Input(shape=(200,300,3),name = 'image_input')

#Use the generated model 
output_vgg16_conv = model_vgg16_conv(ip)

#Add the fully-connected layers 
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(8, activation='softmax', name='predictions')(x)

#Create your own model 
my_model = Model(inputs=ip, outputs=x)

#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
my_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     [(None, 200, 300, 3)]     0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
fc1 (Dense)                  (None, 4096)              113250304 
_________________________________________________________________
fc2 (Dense)                  (None, 4096)              16781312  
_________________________________________________________________
predictions (Dense)          (None, 8)                 32776     
Total params: 144,779,080
Trainable params: 144,779,080
Non-trainable params: 0
_____________________________________________

In [43]:
temp_labels = []
temp_features = []

for index in range(0, len(X_train)):
    img = X_train[index]
    img = preprocess_input(img)
    # Extracting features
    yhat = my_model.predict(img)
    # convert the probabilities to class labels
    label = decode_predictions(yhat)
    # retrieve the most likely result, e.g. highest probability
    label = label[0][0]
    # print the classification
    print('%s (%.2f%%)' % (label[1], label[2]*100))

ValueError: Error when checking input: expected image_input to have shape (200, 300, 3) but got array with shape (224, 224, 3)

In [35]:
from keras import models
base_model = VGG16(weights='imagenet')
model_VGG16 = models.Model(inputs=base_model.input, outputs=base_model.get_layer('flatten').output)

TypeError: object of type 'MaxPooling2D' has no len()